문제 2-3: 학생 정보 구조화 시스템

In [1]:
import os
from typing import List
from pprint import pprint
from dotenv import load_dotenv
from langchain_upstage import ChatUpstage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

# .env 파일에서 환경 변수 로드
load_dotenv()

# LLM 설정 (Upstage Solar 모델 사용)
llm = ChatUpstage(
    api_key=os.getenv("UPSTAGE_API_KEY"),
    model="solar-1-mini-chat", # solar-1-mini-chat-ko-instruct 모델도 사용 가능
    temperature=0.1
)

# 1. Pydantic 모델(데이터 구조) 정의
class StudentInfo(BaseModel):
    name: str = Field(description="학생의 이름")
    age: int = Field(description="학생의 나이")
    major: str = Field(description="학생의 전공")
    hobbies: List[str] = Field(description="학생의 취미 목록")
    goal: str = Field(description="학생의 목표")

# 2. PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object=StudentInfo)

# 3. 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 주어진 텍스트에서 학생 정보를 추출하는 전문가입니다.\n{format_instructions}"),
        ("human", "{introduction}"),
    ]
)

# 4. 체인 생성 및 실행
chain = prompt | llm | parser
introduction_text = "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. 취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다."
result = chain.invoke({
    "introduction": introduction_text,
    "format_instructions": parser.get_format_instructions(),
})

print("## 입력 텍스트:")
pprint(introduction_text)
print("\n## 추출된 구조화 데이터:")
pprint(result.dict())

## 입력 텍스트:
('안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. 취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 '
 '개발자가 되는 것이 목표입니다.')

## 추출된 구조화 데이터:
{'age': 22,
 'goal': '훌륭한 개발자가 되기',
 'hobbies': ['게임하기', '영화보기', '코딩'],
 'major': '컴퓨터공학',
 'name': '김민수'}


C:\Users\user\AppData\Local\Temp\ipykernel_22616\653007045.py:50: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  pprint(result.dict())
